In [27]:
import pandas as pd
import numpy as np

In [28]:
mutual_art = pd.read_csv('../../datasets/clean/clean_mut_art_ver2_ajustado.csv').drop(columns=['Unnamed: 0'])
artsy = pd.read_csv('../../datasets/clean/datosArtsy.csv')

In [29]:
def to_lower(df):
  columnsTolower = list(df.select_dtypes(include=['category','object']))
  for feature in columnsTolower:
      try:
          df[feature] = df[feature].str.lower()
      except:
          print('Error parsing '+feature)
  return df

In [30]:
artsy.height = artsy.height.str.split(' ')
artsy.length = artsy.length.str.split(' ')
artsy.height = artsy.height.apply(lambda x: x[0])
artsy.length = artsy.length.apply(lambda x: x[0])

In [31]:
# Transformación de tipo de datos
df = pd.concat([artsy,mutual_art])
df['height'] = df['height'].astype(float)
df['length'] = df['length'].astype(float)
df['price'] = df['price'].astype(float)
df['performance'] = df['performance'].astype(float)
#df['año_venta'] = pd.to_datetime(df['sale_date'],yearfirst=True)
#df['año_venta'] = df['año_venta'].astype(str).str.slice(stop=4)


# Añadir precio por cm2
df = df.assign(precio_cm2 = lambda x: (x['price']/(x['height']*x['length'])))
#Eliminar valores infinitos (Surgen cuando largo o ancho son 0)
df=df[~df.isin([np.inf, -np.inf]).any(1)]
df=to_lower(df)
df['author'] = df['author'].str.strip()

In [32]:
df['fecha'] = pd.to_datetime(df['fecha'], format="%Y-%m-%d")

In [33]:
df['mes'] = df['fecha'].dt.month.astype(str)
df['año'] = df['fecha'].dt.year.astype(str)

In [34]:
seleccion_artistas = df.groupby(['author']).agg({'año':['nunique','count'],'mes':['nunique','count'],'price':['mean','count']})

In [35]:
seleccion_artistas = seleccion_artistas[seleccion_artistas['año']['nunique']>13].reset_index()

In [36]:
seleccion_artistas

author     año           mes                price      
                            nunique count nunique count           mean count
0    alfredo ramos martínez      22    96      11    96  151186.406250    96
1             angel zárraga      19    68       9    68  103368.014706    68
2             antonio seguí      25   402      12   402   23743.567164   402
3           armando reverón      19    43       4    43  242734.604651    43
4   benito quinquela martin      15   135       9   135   36451.866667   135
5    david alfaro siqueiros      24   787      13   787   35733.521154   780
6              diego rivera      26   811      13   811  114835.617728   810
7          emilio pettoruti      19    94       9    94   78098.602128    94
8       fernando de szyszlo      23   296      12   296   46401.516892   296
9          francisco toledo      25   723      13   723   51985.467479   718
10         francisco zuñiga      24  1181      12  1181   18667.059778  1171
11  graciela rodo-boulanger      16    51      12    51    2916.240000    50
12             héctor poleo      14    20       4    20   51964.700000    20
13        jesus-rafael soto      25   822      12   822   93928.495737   821
14    joaquín torres garcía      24   241      12   241  157177.476793   237
15             juan soriano      20   196      13   196   23726.571990   196
16        mariano rodriguez      22    94       9    94   85018.659574    94
17        rené portocarrero      24   254      12   254   30601.782609   253
18            roberto matta      15  1144      13  1144   55441.066518  1143
19            rufino tamayo      26  3084      13  3084   65723.748785  3071
20            tomás sánchez      24   159      10   159  182194.597484   159

artistas_obras = df[['author','title']].groupby(['author']).agg({'title':['count']})
artistas_obras = artistas_obras['title']
artistas_obras = artistas_obras.sort_values(by='count',ascending=False)
artistas_obras = artistas_obras.reset_index()
lista_artistas = artistas_obras.author.head(27).to_list()
df_artistas = df[df.author.isin(lista_artistas)]
df_artistas['fecha_corta'] = df_artistas['fecha'].str.slice(0,7)+'-01'

In [37]:
lista_artistas = seleccion_artistas.author.to_list()
df['fecha_corta'] = df['año'].astype(str) + '-' + df['mes'].astype(str)
df_artistas = df[df.author.isin(lista_artistas)]

In [38]:
df_artistas

,title,height,length,art_type,medium_text,date_text,age,house,sale_date,price,author,performance,source,country,fecha,precio_cm2,mes,año,fecha_corta
1201,reflejos plateados,69.900,60.000,painting,oil on masonite,1977.0,87.0,christie's,1969-12-31 18:00:44.033,12500.0,benito quinquela martin,-0.170000,artsy,argentina,NaT,2.980448,nan,nan,nan-nan
1202,mañana de invierno con escarcha,121.900,121.900,painting,oil on board,1977.0,87.0,christie's,1969-12-31 18:00:43.789,40000.0,benito quinquela martin,-0.200000,artsy,argentina,NaT,2.691861,nan,nan,nan-nan
1203,tormenta con sol,48.500,59.500,painting,oil on board,1969.0,79.0,sotheby's,1969-12-31 18:00:43.782,17500.0,benito quinquela martin,0.000000,artsy,argentina,NaT,6.064281,nan,nan,nan-nan
1204,contra luiz,89.500,99.700,painting,oil on board,1960.0,70.0,bonham,1969-12-31 18:00:43.599,27575.0,benito quinquela martin,-0.080000,artsy,argentina,NaT,3.090276,nan,nan,nan-nan
1205,puerto de buenos aires,70.000,50.000,painting,oil on board,1977.0,87.0,christie's,1969-12-31 18:00:43.060,25000.0,benito quinquela martin,-0.170000,artsy,argentina,NaT,7.142857,nan,nan,nan-nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10457,senza titolo,252.730,150.000,arte moderna e contemporanea,pastel on paper mounted on canvas,1951.0,71.0,christie's,NaN,68442.0,roberto matta,0.908236,mutual art,-,2008-11-24,1.805405,11.0,2008.0,2008.0-11.0
10472,"no, no dice uno mientras otro duerme en el suelo",190.750,190.750,post-war & contemporary art,graphite and conté crayon and pastel on paper,1966.0,56.0,christie's,NaN,7716.0,roberto matta,0.750000,mutual art,-,2008-12-11,0.212062,12.0,2008.0,2008.0-12.0
10480,las majas desnudas (dessin pour carmen ),28.000,38.100,impressionist & modern art day sale,"wax crayon, coloured crayon and pencil on paper",1940.0,82.0,sotheby's,NaN,37453.0,roberto matta,0.750000,mutual art,-,2009-02-04,35.107799,2.0,2009.0,2009.0-2.0
10491,the new school,6.000,7.875,19th & 20th century prints & drawings,drypoint,1943.0,79.0,swann auction galleries,NaN,2319.0,roberto matta,1.120019,mutual art,-,2009-03-05,49.079365,3.0,2009.0,2009.0-3.0


In [39]:
df_artistas_anual = df_artistas[['author','año','price']]
df_artistas = df_artistas[['author','fecha_corta','price']]

In [40]:
df_artistas['% change'] = df_artistas.groupby(['author'])['price'].pct_change()
df_artistas_anual['% change'] = df_artistas_anual.groupby(['author'])['price'].pct_change()

/var/folders/c7/yrk77vqj3jj31wf8sfvg8tyw0000gn/T/ipykernel_70870/3981855572.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_artistas['% change'] = df_artistas.groupby(['author'])['price'].pct_change()
/var/folders/c7/yrk77vqj3jj31wf8sfvg8tyw0000gn/T/ipykernel_70870/3981855572.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_artistas_anual['% change'] = df_artistas_anual.groupby(['author'])['price'].pct_change()


In [41]:
df_artistas_anual.tail(10)

,author,año,price,% change
10345,roberto matta,2008.0,3577.0,1.599564
10364,roberto matta,2008.0,17195.0,3.807101
10397,roberto matta,2008.0,68780.0,3.000000
10416,roberto matta,2008.0,27512.0,-0.600000
10456,roberto matta,2008.0,57272.0,1.081710
10457,roberto matta,2008.0,68442.0,0.195034
10472,roberto matta,2008.0,7716.0,-0.887262
10480,roberto matta,2009.0,37453.0,3.853940
10491,roberto matta,2009.0,2319.0,-0.938082
10492,roberto matta,2009.0,2651.0,0.143165


In [42]:
len(df_artistas.author.unique())

21

In [43]:
df_artistas = df_artistas.groupby(['author','fecha_corta']).agg({'% change':['mean']})
df_artistas_anual = df_artistas_anual.groupby(['author','año']).agg({'% change':['mean']})

In [44]:
df_artistas_anual

% change
                                    mean
author                 año              
alfredo ramos martínez 2000.0        NaN
                       2001.0   1.543487
                       2002.0  -0.301231
                       2003.0  -0.155540
                       2004.0   1.819231
...                                  ...
tomás sánchez          2018.0   2.481098
                       2019.0  10.913080
                       2020.0  17.510908
                       2021.0  13.280020
                       2022.0  89.535586

[451 rows x 1 columns]

In [45]:
df_artistas = df_artistas['% change'].reset_index()
df_artistas_anual = df_artistas_anual['% change'].reset_index()

In [46]:
df_artistas_wide = df_artistas.pivot(index='fecha_corta', columns='author', values='mean').reset_index()
df_artistas_anual_wide = df_artistas_anual.pivot(index='año', columns='author', values='mean').reset_index()

In [47]:
df_artistas_anual_wide

author,año,alfredo ramos martínez,angel zárraga,antonio seguí,armando reverón,benito quinquela martin,david alfaro siqueiros,diego rivera,emilio pettoruti,fernando de szyszlo,...,graciela rodo-boulanger,héctor poleo,jesus-rafael soto,joaquín torres garcía,juan soriano,mariano rodriguez,rené portocarrero,roberto matta,rufino tamayo,tomás sánchez
0,1998.0,NaN,NaN,NaN,NaN,NaN,NaN,0.002920,NaN,NaN,...,NaN,NaN,17.722892,-0.124946,NaN,NaN,-0.227428,45.097508,1.296444,NaN
1,1999.0,NaN,-0.778693,1.271224,NaN,NaN,NaN,38.179396,3.914348,3.362000,...,NaN,NaN,3.300723,8.751479,NaN,1.181067,0.741738,1.232232,14.228287,1.500000
2,2000.0,NaN,1.794673,10.579036,1.341353,NaN,1.802350,0.317234,NaN,NaN,...,NaN,NaN,13.817426,NaN,NaN,NaN,5.325215,2.342877,5.376149,1.906532
3,2001.0,1.543487,1.107088,-0.954184,-0.298801,NaN,17.016052,11.875910,NaN,-0.279092,...,NaN,NaN,9.362430,-0.502144,NaN,-0.218975,NaN,6.077721,28.055561,0.678322
4,2002.0,-0.301231,NaN,2.301788,0.617561,NaN,4.192325,3.488484,NaN,-0.322533,...,-0.874715,-0.639140,0.763869,59.570266,NaN,10.550642,-0.381364,5.532216,6.264322,0.325690
5,2003.0,-0.155540,3.823754,0.446940,NaN,NaN,26.647831,0.815081,NaN,0.193171,...,NaN,0.303754,-0.264183,-0.600376,0.833161,0.300339,0.318457,11.918002,3.676416,0.721117
6,2004.0,1.819231,1.984529,1.340978,NaN,NaN,-0.067445,1.675534,NaN,NaN,...,NaN,NaN,2.700006,0.947893,NaN,NaN,0.704094,3.927280,21.762015,0.920809
7,2005.0,-0.788098,-0.012382,2.382011,NaN,NaN,8.055694,1.036019,-0.907997,0.655329,...,87.236364,NaN,24.753476,3.499032,-0.113212,-0.519695,1.601801,6.060535,17.673087,2.293562
8,2006.0,373.092207,3.853103,0.328354,-0.377866,NaN,0.534711,1.948648,NaN,-0.411826,...,NaN,NaN,2.284741,7.443287,-0.272916,3.674981,0.586301,2.880377,0.936248,1.399481
9,2007.0,0.629564,0.651124,2.438945,0.562848,NaN,10.270163,1.201857,15.522378,-0.106369,...,NaN,NaN,25.038514,1.192855,5.483851,20.450832,10.738840,12.623469,2.403353,0.547940


In [48]:
#df_artistas_wide.to_excel('../../datasets/clean/performance_artistas_pct_chsnge.xlsx')
df_artistas_anual_wide.to_excel('../../datasets/clean/performance_anual_artistas_pct_chsnge.xlsx')